# Tutorial :: Last week homework

Your next task is analyse the data from a different perspective using the same dataset. Which way would be fair to analyse the data?

### Your insights

???

---

## Tutorial :: The power of the news

**CONCERN**

You are working for a consultancy firm in charge of political image. Last year Australia had a problem with France for a submarine deal that was pulled off. A report was already generated with the news titles. However, your job as an analyst is to generate a thorough report with additional information inside each news.

1. **Q**uestion
2. **D**ata
3. **A**nalysis
4. **V**isualisation
5. **I**nsight

<img src="graphics/QDAVI_cycle_sm.png" width="50%" />

### 1. Question

How the news affected the image of the government?

**Tip:** You can combine web scrapping and APIs

### 2. Data

You must use The Guardian API

**Tip:** Check the studio session for information about how to call the guardian API

In [26]:
# Libraries for the analysis
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from bs4 import BeautifulSoup
import urllib.request

In [27]:
# Build a search URL
baseUrl = 'https://content.guardianapis.com/search?q=' # content search

searchString = "submarine"
office = "&production-office=aus"
tag = "&tag=politics/politics"
fromDate = "&from-date=2021-09-01"

url = baseUrl+'"'+searchString+'"'+office+fromDate+"&api-key=test"
print(url)

https://content.guardianapis.com/search?q="submarine"&production-office=aus&from-date=2021-09-01&api-key=test


In [28]:
# Call the API
response = requests.get(url)
data = json.loads(response.content)
results = data['response']['results']
results

[{'id': 'world/2021/nov/09/australia-promises-jobs-to-workers-stranded-by-scrapping-of-french-submarine-deal',
  'type': 'article',
  'sectionId': 'world',
  'sectionName': 'World news',
  'webPublicationDate': '2021-11-08T16:30:35Z',
  'webTitle': 'Australia promises jobs to workers stranded by scrapping of French submarine deal',
  'webUrl': 'https://www.theguardian.com/world/2021/nov/09/australia-promises-jobs-to-workers-stranded-by-scrapping-of-french-submarine-deal',
  'apiUrl': 'https://content.guardianapis.com/world/2021/nov/09/australia-promises-jobs-to-workers-stranded-by-scrapping-of-french-submarine-deal',
  'isHosted': False,
  'pillarId': 'pillar/news',
  'pillarName': 'News'},
 {'id': 'world/2021/oct/01/fears-australias-france-submarine-snub-could-scupper-closer-eu-economic-ties',
  'type': 'article',
  'sectionId': 'world',
  'sectionName': 'World news',
  'webPublicationDate': '2021-10-01T09:00:58Z',
  'webTitle': 'Fears Australia’s France submarine snub could scupper c

The results contain the URL to the news website. After inspecting a couple of pages, which information could be eaiser to extrat from it

In [30]:
# Get HTML function
def get_HTML(url):
    response = urllib.request.urlopen(url)
    html = response.read()
    return html

In [34]:
# Beautiful soup function for subtitle
def extract_subTitle(HTML):
    soup = BeautifulSoup(HTML, "html.parser") # the html input and the parser name
    article = soup.find("article") # the tag that contains the article
    div_element = article.find("div", attrs={"data-gu-name": "standfirst"}) # the tag that can be found using an attribute
    target_element = div_element.find("p")
    return target_element.text

In [40]:
# Beautiful soup function for body
def extract_body(HTML):
    soup = BeautifulSoup(HTML, "html.parser") # the html input and the parser name
    article = soup.find("article") # the tag that contains the article
    div_element = article.find("div", attrs={"id": "maincontent"}) # the tag that can be found using an attribute
    div_div_element = div_element.find("div")
    target_elements = div_element.findAll("p")
    result = ""
    for te in target_elements:
        result += te.text
    return result

#### Clean/preprocess data

In [44]:
# Create a dataframe
df = pd.DataFrame(columns=["Date", "Section", "Title", "Subtitle", "Body"])
df

,Date,Section,Title,Subtitle,Body


In [45]:
# Populate the dataframe
for news in results:
    html = get_HTML(news["webUrl"])
    data = {"Date": news["webPublicationDate"], "Section": news["sectionName"], "Title": news["webTitle"], "Subtitle": extract_subTitle(html), "Body": extract_body(html)}
    df = df.append(data, ignore_index=True)
df

,Date,Section,Title,Subtitle,Body
0,2021-11-08T16:30:35Z,World news,Australia promises jobs to workers stranded by...,Defence industry minister Melissa Price tells ...,“Each and every” skilled shipbuilding worker a...
1,2021-10-01T09:00:58Z,World news,Fears Australia’s France submarine snub could ...,Opposition accuses Scott Morrison of failing ‘...,The postponement of trade talks between the Eu...
2,2021-09-29T06:15:33Z,Australia news,Australia tore up French submarine contract ‘f...,Shipbuilding company maintains it ‘did not fai...,Australia scrapped the $90bn submarine deal wi...
3,2021-11-18T16:30:10Z,World news,‘Naughty guy’: top Chinese diplomat accuses Au...,"Exclusive: Acting ambassador to Australia, Wan...",A top Chinese diplomat has likened Australia t...
4,2021-10-28T04:39:00Z,World news,Australia’s foreign minister to meet French am...,Marise Payne says she regrets France’s ‘deep d...,Australia’s foreign minister will meet with th...
5,2021-09-29T02:50:32Z,World news,Former US navy secretary now Scott Morrison’s ...,"Prof Donald Winter, who advised the Australian...",A former US navy secretary who advised the Aus...
6,2021-10-29T04:32:28Z,Australia news,Macron’s anger over nuclear submarine deal lin...,Australian defence minister’s claim comes as F...,Peter Dutton says sustained expressions of out...
7,2021-09-29T08:19:43Z,Australia news,Malcolm Turnbull excoriates Scott Morrison ove...,Former PM also revealed he plans to go to Glas...,Malcolm Turnbull has revealed he has spoken to...
8,2021-09-20T01:11:38Z,World news,‘We felt fooled’: France still furious after A...,"‘Maybe we’re not friends,’ recalled ambassador...",French anger at the Morrison government’s deci...
9,2021-09-17T08:35:30Z,Australia news,Game-changer or irresponsible? The known unkno...,Analysis: Stay tuned as we battle the notoriou...,"It started, as defence stories so often do, wi..."


### 3. Analysis

Information extraction?

### 4. Visualisation

### 5. Insights
